# Insert, Update and Delete data on S3 Datalake using Apache Hudi and Amazon EMR - Demo

## Table of Contents:

1. [Overview](#Overview)
2. [Copy On Write](#Copy-On-Write)<br>
    2.1 [Bulk Insert the Initial Dataset](#Bulk-Insert-the-Initial-Dataset)<br>
    2.2 [Batch Upsert some records](#Batch-Upsert-some-records)<br>
    2.3 [Deleting Records](#Deleting-Records)<br>
3. [Rollback](#Rollback)<br>
4. [Time travel with Hudi](#Time-travel-with-Hudi) 
5. [Merge on Read](#Merge-On-Read)<br>
    5.1 [Bulk Insert the Initial Dataset](#Bulk-Insert-the-Initial-Dataset)<br>
    5.2 [Batch Upsert some records](#Batch-Upsert-some-records)<br>
    5.3 [Compaction for MOR tables](#Compaction-for-MOR-tables) <br>
6. [Working with Partitioned Tables](#Working-with-Partitioned-Tables)

## Overview

This notebook demonstrates using PySpark on [Apache Hudi](https://aws.amazon.com/emr/features/hudi/) on Amazon EMR to insert/upsert/delete records to an S3 data lake.

Here are some good reference links to read later:

* [Apache Hudi concepts](https://hudi.apache.org/concepts.html)
* [How Hudi Works](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-hudi-how-it-works.html)

This notebook covers the following concepts when writing Copy-On-Write and Merge-On-Read tables to an S3 Datalake:

- Write Hudi Spark jobs in PySpark.
- Bulk Insert the Initial Dataset.
- Write a MultiKey Partitioned table as well as a Non-Partitioned table.
- Tune the Bulk Insert write performance as per expected number of target files.
- Sync the Hudi tables to the Hive/Glue Catalog.
- Upsert some records to a Hudi table.
- Delete from records from a Hudi table.
- Understand how Hudi Commit Retention policy works.
- Time travel with Hudi incremental tables using incremental and point-in-time queries
- Perform Insert/Upsert/Delete operations for Merge-on-Read table and understand the difference between MOR and COW tables


#### This demo runs fine on a 3 node (r4.8xlarge) EMR release 6.2 cluster.

Let's start by initializing the Spark Session to connect this notebook to our Spark EMR cluster:

Note that the files hudi-spark-bundle.jar and spark-avro.jar are copied into HDFS.

hadoop fs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar hdfs:///user/hadoop/

hadoop fs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar hdfs:///user/hadoop/

hadoop fs -copyFromLocal /usr/lib/spark/jars/httpclient-4.5.9.jar hdfs:///user/hadoop/

hadoop fs -copyFromLocal /usr/lib/spark/jars/httpcore-4.4.11.jar hdfs:///user/hadoop/


In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"hdfs:///user/hadoop/httpcore-4.4.11.jar,hdfs:///user/hadoop/httpclient-4.5.9.jar,hdfs:///user/hadoop/hudi-spark-bundle.jar,hdfs:///user/hadoop/spark-avro.jar",
             "spark.sql.hive.convertMetastoreParquet":"false", 
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "24544M",
             "spark.executor.cores": 4,
             "spark.dynamicAllocation.initialExecutors":8
           } 
}

## Configuration

Following are the configuration items we will use in this demo

Make sure to point the target parameter to your S3 bucket (replace <your-bucket> with an S3 bucket name in your AWS account in the same region as your EMR cluster)

In [2]:
## CHANGE ME ##
config = {
    "table_name": "hudi_trips_table",
    "target": "s3://vasveena-test-demo/demos/hudi/hudi_trips_table",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "10"
}

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1633720478106_0037,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The constants for Python to use:

In [9]:
# General Constants
HUDI_FORMAT = "org.apache.hudi"
TABLE_NAME = "hoodie.table.name"
RECORDKEY_FIELD_OPT_KEY = "hoodie.datasource.write.recordkey.field"
PRECOMBINE_FIELD_OPT_KEY = "hoodie.datasource.write.precombine.field"
OPERATION_OPT_KEY = "hoodie.datasource.write.operation"
BULK_INSERT_OPERATION_OPT_VAL = "bulk_insert"
UPSERT_OPERATION_OPT_VAL = "upsert"
BULK_INSERT_PARALLELISM = "hoodie.bulkinsert.shuffle.parallelism"
UPSERT_PARALLELISM = "hoodie.upsert.shuffle.parallelism"
S3_CONSISTENCY_CHECK = "hoodie.consistency.check.enabled"
HUDI_CLEANER_POLICY = "hoodie.cleaner.policy"
KEEP_LATEST_COMMITS = "KEEP_LATEST_COMMITS"
HUDI_COMMITS_RETAINED = "hoodie.cleaner.commits.retained"
PAYLOAD_CLASS_OPT_KEY = "hoodie.datasource.write.payload.class"
EMPTY_PAYLOAD_CLASS_OPT_VAL = "org.apache.hudi.common.model.EmptyHoodieRecordPayload"

# Hive Constants
HIVE_SYNC_ENABLED_OPT_KEY="hoodie.datasource.hive_sync.enable"
HIVE_PARTITION_FIELDS_OPT_KEY="hoodie.datasource.hive_sync.partition_fields"
HIVE_ASSUME_DATE_PARTITION_OPT_KEY="hoodie.datasource.hive_sync.assume_date_partitioning"
HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY="hoodie.datasource.hive_sync.partition_extractor_class"
HIVE_TABLE_OPT_KEY="hoodie.datasource.hive_sync.table"

# Partition Constants
NONPARTITION_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.NonPartitionedExtractor"
MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.MultiPartKeysValueExtractor"
KEYGENERATOR_CLASS_OPT_KEY="hoodie.datasource.write.keygenerator.class"
NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.keygen.NonpartitionedKeyGenerator"
COMPLEX_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.ComplexKeyGenerator"
PARTITIONPATH_FIELD_OPT_KEY="hoodie.datasource.write.partitionpath.field"

#Incremental Constants
VIEW_TYPE_OPT_KEY="hoodie.datasource.view.type"
BEGIN_INSTANTTIME_OPT_KEY="hoodie.datasource.read.begin.instanttime"
VIEW_TYPE_INCREMENTAL_OPT_VAL="incremental"
END_INSTANTTIME_OPT_KEY="hoodie.datasource.read.end.instanttime"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Functions to create JSON data and Spark dataframe from this data

In [6]:
## Generates Data

from datetime import datetime

def get_json_data(start, count, dest):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data = [{"trip_id": i, "tstamp": time_stamp, "route_id": chr(65 + (i % 10)), "destination": dest[i%10]} for i in range(start, start + count)]
    return data

# Creates the Dataframe
def create_json_df(spark, data):
    sc = spark.sparkContext
    return spark.read.json(sc.parallelize(data))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Bulk Insert the Initial Dataset

Let's generate 2M records to load into our Data Lake:

In [5]:
dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df1 = create_json_df(spark, get_json_data(0, 2000000, dest))
print(df1.count())
df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2000000
+-------------+--------+-------+-------------------+
|  destination|route_id|trip_id|             tstamp|
+-------------+--------+-------+-------------------+
|      Seattle|       A|      0|2021-10-10 23:50:43|
|     New York|       B|      1|2021-10-10 23:50:43|
|   New Jersey|       C|      2|2021-10-10 23:50:43|
|  Los Angeles|       D|      3|2021-10-10 23:50:43|
|    Las Vegas|       E|      4|2021-10-10 23:50:43|
|       Tucson|       F|      5|2021-10-10 23:50:43|
|Washington DC|       G|      6|2021-10-10 23:50:43|
| Philadelphia|       H|      7|2021-10-10 23:50:43|
|        Miami|       I|      8|2021-10-10 23:50:43|
|San Francisco|       J|      9|2021-10-10 23:50:43|
|      Seattle|       A|     10|2021-10-10 23:50:43|
|     New York|       B|     11|2021-10-10 23:50:43|
|   New Jersey|       C|     12|2021-10-10 23:50:43|
|  Los Angeles|       D|     13|2021-10-10 23:50:43|
|    Las Vegas|       E|     14|2021-10-10 23:50:43|
|       Tucson|       F|     15|2021-1

And write the data to S3:

In [6]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .mode("Overwrite")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Let's observe the number of files in S3. Expected number of files is 3 files as BULK_INSERT_PARALLELISM is set to 3. 
```

$ aws s3 ls s3://<your-bucket>/demos/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2021-10-10 23:51:57    0 Bytes .hoodie_$folder$
2021-10-10 23:53:40   93 Bytes .hoodie_partition_metadata
2021-10-10 23:53:50    4.8 MiB 128bbd02-25e8-4aa1-a651-3d1b20227f77-0_1-8-100_20211010235153.parquet
2021-10-10 23:53:48    4.4 MiB 59043700-7191-4248-bb28-44f4f314e9d0-0_0-8-99_20211010235153.parquet
2021-10-10 23:53:49    4.6 MiB fff0690b-28ca-4a63-8a08-b7d97020a802-0_2-8-101_20211010235153.parquet

Total Objects: 5
   Total Size: 13.8 MiB

```

```

Let's inspect the table created and query the data.

In [7]:
spark.sql("show tables").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------------------+-----------+
|database|tableName                     |isTemporary|
+--------+------------------------------+-----------+
|default |amazon_reviews_parquet        |false      |
|default |big1vang1                     |false      |
|default |big1vang1_1                   |false      |
|default |big2vang2                     |false      |
|default |big2vang2_1                   |false      |
|default |chopsticks_insert             |false      |
|default |chopsticks_orcc               |false      |
|default |chopsticks_orcc2              |false      |
|default |chopsticks_orcc_athena        |false      |
|default |chopsticks_orcc_athena2       |false      |
|default |chopsticks_orcc_athena4       |false      |
|default |chopsticks_orcc_athena6       |false      |
|default |chopsticks_orcc_dest          |false      |
|default |chopsticks_orcc_temp2         |false      |
|default |chopsticks_xlsx               |false      |
|default |cloudtrail_logs_va

Check the Glue data catalog for a new Hudi table called hudi_trips_table. Check the schema and notice the new columns added by Hudi to keep track of commits and filenames

In [8]:
df2=spark.read.format(HUDI_FORMAT).load(config["target"]+"/*")
df2.count()
df2.show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+---------------------------------------------------------------------+-------------+--------+-------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name                                                    |destination  |route_id|trip_id|tstamp             |
+-------------------+--------------------+------------------+----------------------+---------------------------------------------------------------------+-------------+--------+-------+-------------------+
|20211010235153     |20211010235153_1_1  |1572203           |                      |128bbd02-25e8-4aa1-a651-3d1b20227f77-0_1-8-100_20211010235153.parquet|Los Angeles  |D       |1572203|2021-10-10 23:50:43|
|20211010235153     |20211010235153_1_2  |1572204           |                      |128bbd02-25e8-4aa1-a651-3d1b20227f77-0_1-8-100_20211010235153.parquet|Las Vegas    |E       

We can query the Hive table as well:

In [9]:
spark.sql("select count(*) from "+config['table_name']).show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|2000000 |
+--------+

### Batch Upsert some records

Let's modify a few records:

In [10]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name'] +" where trip_id between 1000000 and 1000009").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|1000000|A       |Seattle      |2021-10-10 23:50:43|
|1000001|B       |New York     |2021-10-10 23:50:43|
|1000002|C       |New Jersey   |2021-10-10 23:50:43|
|1000003|D       |Los Angeles  |2021-10-10 23:50:43|
|1000004|E       |Las Vegas    |2021-10-10 23:50:43|
|1000005|F       |Tucson       |2021-10-10 23:50:43|
|1000006|G       |Washington DC|2021-10-10 23:50:43|
|1000007|H       |Philadelphia |2021-10-10 23:50:43|
|1000008|I       |Miami        |2021-10-10 23:50:43|
|1000009|J       |San Francisco|2021-10-10 23:50:43|
+-------+--------+-------------+-------------------+

In [11]:
upsert_dest = ["Boston", "Boston", "Boston", "Boston", "Boston","Boston","Boston","Boston","Boston","Boston"]
df3 = create_json_df(spark, get_json_data(1000000, 10, upsert_dest))
df3.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10

In [12]:
df3.select("trip_id","route_id","tstamp","destination").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------------+-----------+
|trip_id|route_id|             tstamp|destination|
+-------+--------+-------------------+-----------+
|1000000|       A|2021-10-10 23:58:01|     Boston|
|1000001|       B|2021-10-10 23:58:01|     Boston|
|1000002|       C|2021-10-10 23:58:01|     Boston|
|1000003|       D|2021-10-10 23:58:01|     Boston|
|1000004|       E|2021-10-10 23:58:01|     Boston|
|1000005|       F|2021-10-10 23:58:01|     Boston|
|1000006|       G|2021-10-10 23:58:01|     Boston|
|1000007|       H|2021-10-10 23:58:01|     Boston|
|1000008|       I|2021-10-10 23:58:01|     Boston|
|1000009|       J|2021-10-10 23:58:01|     Boston|
+-------+--------+-------------------+-----------+

We have changed the destination and timestamp for trip IDs 1000000 to 1000010. Now, let's upsert the changes to S3. Note that the operation now is "Upsert" as opposed to BulkInsert for the initial load:

```
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)

```

In [13]:
(df3.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id between 999996 and 1000013").show(50,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------------+-------------+
|trip_id|route_id|tstamp             |destination  |
+-------+--------+-------------------+-------------+
|1000000|A       |2021-10-10 23:58:01|Boston       |
|1000001|B       |2021-10-10 23:58:01|Boston       |
|1000002|C       |2021-10-10 23:58:01|Boston       |
|1000003|D       |2021-10-10 23:58:01|Boston       |
|1000004|E       |2021-10-10 23:58:01|Boston       |
|1000005|F       |2021-10-10 23:58:01|Boston       |
|1000006|G       |2021-10-10 23:58:01|Boston       |
|1000007|H       |2021-10-10 23:58:01|Boston       |
|1000008|I       |2021-10-10 23:58:01|Boston       |
|1000009|J       |2021-10-10 23:58:01|Boston       |
|1000010|A       |2021-10-10 23:50:43|Seattle      |
|1000011|B       |2021-10-10 23:50:43|New York     |
|1000012|C       |2021-10-10 23:50:43|New Jersey   |
|1000013|D       |2021-10-10 23:50:43|Los Angeles  |
|999996 |G       |2021-10-10 23:50:43|Washington DC|
|999997 |H       |2021-10-10 23:50:43|Philadel

Now lets check the commit

```
$ aws s3 ls s3://<your-bucket>/demos/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2021-10-10 23:51:57    0 Bytes .hoodie_$folder$
2021-10-10 23:53:40   93 Bytes .hoodie_partition_metadata
2021-10-10 23:53:50    4.8 MiB 128bbd02-25e8-4aa1-a651-3d1b20227f77-0_1-8-100_20211010235153.parquet
2021-10-10 23:58:45    4.4 MiB 59043700-7191-4248-bb28-44f4f314e9d0-0_0-55-538_20211010235834.parquet
2021-10-10 23:53:48    4.4 MiB 59043700-7191-4248-bb28-44f4f314e9d0-0_0-8-99_20211010235153.parquet
2021-10-10 23:53:49    4.6 MiB fff0690b-28ca-4a63-8a08-b7d97020a802-0_2-8-101_20211010235153.parquet

Total Objects: 6
   Total Size: 18.2 MiB

```

Now that we upserted some records, let us try to insert 10 new records into the table. We will use same upsert option. As primary keys 2000000 to 2000009 do not exist in the table, the records will be inserted. 

In [15]:
insert_dest = ["Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse"]
df5 = create_json_df(spark, get_json_data(2000000, 10, insert_dest))
df5.count()
df5.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+-------+-------------------+
|destination|route_id|trip_id|             tstamp|
+-----------+--------+-------+-------------------+
|   Syracuse|       A|2000000|2021-10-10 23:59:44|
|   Syracuse|       B|2000001|2021-10-10 23:59:44|
|   Syracuse|       C|2000002|2021-10-10 23:59:44|
|   Syracuse|       D|2000003|2021-10-10 23:59:44|
|   Syracuse|       E|2000004|2021-10-10 23:59:44|
|   Syracuse|       F|2000005|2021-10-10 23:59:44|
|   Syracuse|       G|2000006|2021-10-10 23:59:44|
|   Syracuse|       H|2000007|2021-10-10 23:59:44|
|   Syracuse|       I|2000008|2021-10-10 23:59:44|
|   Syracuse|       J|2000009|2021-10-10 23:59:44|
+-----------+--------+-------+-------------------+

In [16]:
(df5.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df6=spark.read.format(HUDI_FORMAT).load(config["target"]+"/*")
df6.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2000010

In [18]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(50,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------------+-------------+
|trip_id|route_id|tstamp             |destination  |
+-------+--------+-------------------+-------------+
|2000009|J       |2021-10-10 23:59:44|Syracuse     |
|2000008|I       |2021-10-10 23:59:44|Syracuse     |
|2000007|H       |2021-10-10 23:59:44|Syracuse     |
|2000006|G       |2021-10-10 23:59:44|Syracuse     |
|2000001|B       |2021-10-10 23:59:44|Syracuse     |
|2000000|A       |2021-10-10 23:59:44|Syracuse     |
|2000005|F       |2021-10-10 23:59:44|Syracuse     |
|2000004|E       |2021-10-10 23:59:44|Syracuse     |
|2000003|D       |2021-10-10 23:59:44|Syracuse     |
|2000002|C       |2021-10-10 23:59:44|Syracuse     |
|1999997|H       |2021-10-10 23:50:43|Philadelphia |
|1999998|I       |2021-10-10 23:50:43|Miami        |
|1999999|J       |2021-10-10 23:50:43|San Francisco|
+-------+--------+-------------------+-------------+

We can observe that the records are updated.

## Deleting Records.

Apache Hudi supports implementing two types of deletes on data stored in Hudi datasets, by enabling the user to specify a different record payload implementation.

* **Soft Deletes** : With soft deletes, user wants to retain the key but just null out the values for all other fields. This can be simply achieved by ensuring the appropriate fields are nullable in the dataset schema and simply upserting the dataset after setting these fields to null.
    
* **Hard Deletes** : A stronger form of delete is to physically remove any trace of the record from the dataset. 

Let's now execute some hard delete operations on our dataset which will remove the records from our dataset.

Let's delete the 10 records with the "Syracuse" destination we added to the table. Note that the only change is the single line that set the hoodie.datasource.write.payload.class to org.apache.hudi.EmptyHoodieRecordPayload to delete the records.

```
.option(PAYLOAD_CLASS_OPT_KEY, EMPTY_PAYLOAD_CLASS_OPT_VAL)
```

In [19]:
df5.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+-------+-------------------+
|destination|route_id|trip_id|             tstamp|
+-----------+--------+-------+-------------------+
|   Syracuse|       A|2000000|2021-10-10 23:59:44|
|   Syracuse|       B|2000001|2021-10-10 23:59:44|
|   Syracuse|       C|2000002|2021-10-10 23:59:44|
|   Syracuse|       D|2000003|2021-10-10 23:59:44|
|   Syracuse|       E|2000004|2021-10-10 23:59:44|
|   Syracuse|       F|2000005|2021-10-10 23:59:44|
|   Syracuse|       G|2000006|2021-10-10 23:59:44|
|   Syracuse|       H|2000007|2021-10-10 23:59:44|
|   Syracuse|       I|2000008|2021-10-10 23:59:44|
|   Syracuse|       J|2000009|2021-10-10 23:59:44|
+-----------+--------+-------+-------------------+

In [20]:
(df5.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .option(PAYLOAD_CLASS_OPT_KEY, EMPTY_PAYLOAD_CLASS_OPT_VAL)
      .mode("Append")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------------+-------------+
|trip_id|route_id|tstamp             |destination  |
+-------+--------+-------------------+-------------+
|1999997|H       |2021-10-10 23:50:43|Philadelphia |
|1999998|I       |2021-10-10 23:50:43|Miami        |
|1999999|J       |2021-10-10 23:50:43|San Francisco|
+-------+--------+-------------------+-------------+

We can observe that the records > 2000000 no longer exist in our table.

Let's observe the number of files in S3. Expected : 6 files (initial files (3) + one upsert + one insert + one delete = 6)

```

$ aws s3 ls s3://<your-bucket>/demos/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2020-04-28 23:11:39    0 Bytes .hoodie_$folder$
2020-04-28 23:11:52   93 Bytes .hoodie_partition_metadata
2020-04-28 23:15:43    4.8 MiB 0eeafaf2-0110-4056-b509-724b11a4c0b6-0_0-51-609_20200428231528.parquet
2020-04-28 23:11:59    4.8 MiB 0eeafaf2-0110-4056-b509-724b11a4c0b6-0_0-7-67_20200428231141.parquet
2020-04-28 23:18:24    4.4 MiB 1ce9cbac-56e4-477e-a2bc-33fe62b3550f-0_0-135-1376_20200428231814.parquet
2020-04-28 23:17:17    4.4 MiB 1ce9cbac-56e4-477e-a2bc-33fe62b3550f-0_0-93-1007_20200428231705.parquet
2020-04-28 23:11:59    4.4 MiB 1ce9cbac-56e4-477e-a2bc-33fe62b3550f-0_1-7-68_20200428231141.parquet
2020-04-28 23:11:59    4.6 MiB 579443dc-8f56-4990-bed6-a527f21e9682-0_2-7-69_20200428231141.parquet

Total Objects: 8
   Total Size: 27.5 MiB

```

In our example, we set number of commits to retain as 10. So, maximum only 10 new files can be created on top of our bulk insert files. i.e., 13 files in total. If we had set the commits_to_retain as 2, the number of files created will not increase beyond initial files(3) + commits_to_retain(2) = 5 files. This is because Hudi Cleaning Policy will delete older files when writing based on the commit retain policy.

## Rollback

Let's say we want to roll back the last delete we made. 

/usr/lib/hudi/cli/bin/hudi-cli.sh

connect --path s3://<your-bucket>/demos/hudi/hudi_trips_table/

commits show

hudi:hudi_trips_table->commits show
2021-10-11 00:02:55,419 INFO timeline.HoodieActiveTimeline: Loaded instants [[20211010235153__commit__COMPLETED], [20211010235834__commit__COMPLETED], [20211010235951__commit__COMPLETED], [20211011000109__commit__COMPLETED]]
2021-10-11 00:02:55,439 INFO s3n.S3NativeFileSystem: Opening 's3://vasveena-test-demo/demos/hudi/hudi_trips_table/.hoodie/20211011000109.commit' for reading
2021-10-11 00:02:55,747 INFO s3n.S3NativeFileSystem: Opening 's3://vasveena-test-demo/demos/hudi/hudi_trips_table/.hoodie/20211010235951.commit' for reading
2021-10-11 00:02:55,829 INFO s3n.S3NativeFileSystem: Opening 's3://vasveena-test-demo/demos/hudi/hudi_trips_table/.hoodie/20211010235834.commit' for reading
2021-10-11 00:02:55,907 INFO s3n.S3NativeFileSystem: Opening 's3://vasveena-test-demo/demos/hudi/hudi_trips_table/.hoodie/20211010235153.commit' for reading
╔════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime     │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠════════════════╪═════════════════════╪═══════════════════╪═════════════════════╪══════════════════════════╪═══════════════════════╪══════════════════════════════╪══════════════╣
║ 20211011000109 │ 4.4 MB              │ 0                 │ 1                   │ 1                        │ 635786                │ 0                            │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20211010235951 │ 4.4 MB              │ 0                 │ 1                   │ 1                        │ 635796                │ 0                            │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20211010235834 │ 4.4 MB              │ 0                 │ 1                   │ 1                        │ 635786                │ 10                           │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20211010235153 │ 13.8 MB             │ 3                 │ 0                   │ 1                        │ 2000000               │ 0                            │ 0            ║
╚════════════════╧═════════════════════╧═══════════════════╧═════════════════════╧══════════════════════════╧═══════════════════════╧══════════════════════════════╧══════════════╝

commit rollback --commit 20211011000109
    
Now let us check what happened to the records we deleted earlier.

In [22]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------------+-------------+
|trip_id|route_id|tstamp             |destination  |
+-------+--------+-------------------+-------------+
|2000009|J       |2021-10-10 23:59:44|Syracuse     |
|2000008|I       |2021-10-10 23:59:44|Syracuse     |
|2000007|H       |2021-10-10 23:59:44|Syracuse     |
|2000006|G       |2021-10-10 23:59:44|Syracuse     |
|2000001|B       |2021-10-10 23:59:44|Syracuse     |
|2000000|A       |2021-10-10 23:59:44|Syracuse     |
|2000005|F       |2021-10-10 23:59:44|Syracuse     |
|2000004|E       |2021-10-10 23:59:44|Syracuse     |
|2000003|D       |2021-10-10 23:59:44|Syracuse     |
|2000002|C       |2021-10-10 23:59:44|Syracuse     |
|1999997|H       |2021-10-10 23:50:43|Philadelphia |
|1999998|I       |2021-10-10 23:50:43|Miami        |
|1999999|J       |2021-10-10 23:50:43|San Francisco|
+-------+--------+-------------------+-------------+

### Time Travel with Hudi 

Now, let us time travel with Hudi (query previous commits) with incremental and point-in-time queries

First, lets check out incremental queries

In EMR cluster, let us check Hudi CLI and check how commits look right now 

/usr/lib/hudi/cli/bin/hudi-cli.sh

connect --path s3://<your-bucket>/demos/hudi/hudi_trips_table/

commits show

In [23]:
spark.sql("select distinct(_hoodie_commit_time) as commitTime from hudi_trips_table order by commitTime").show(20, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|commitTime    |
+--------------+
|20211010235153|
|20211010235834|
|20211010235951|
+--------------+

In [24]:
commits = spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_trips_table order by commitTime").collect()
print("commits: ")
for elem in commits: print (elem) 

beginTime = commits[-3][0] # commit time we are interested in
print("begin time: " + beginTime)

# incrementally query data
incViewDF = spark.read.format("org.apache.hudi") \
                   .option(VIEW_TYPE_OPT_KEY, VIEW_TYPE_INCREMENTAL_OPT_VAL) \
                   .option(BEGIN_INSTANTTIME_OPT_KEY, beginTime) \
                   .load(config["target"]) 

incViewDF.show(5)
incViewDF.registerTempTable("hudi_incr_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

commits: 
Row(commitTime='20211010235153')
Row(commitTime='20211010235834')
Row(commitTime='20211010235951')
begin time: 20211010235153
+-------------------+--------------------+------------------+----------------------+--------------------+-----------+--------+-------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|destination|route_id|trip_id|             tstamp|
+-------------------+--------------------+------------------+----------------------+--------------------+-----------+--------+-------+-------------------+
|     20211010235834|20211010235834_0_...|           1000000|                      |59043700-7191-424...|     Boston|       A|1000000|2021-10-10 23:58:01|
|     20211010235834|20211010235834_0_...|           1000001|                      |59043700-7191-424...|     Boston|       B|1000001|2021-10-10 23:58:01|
|     20211010235834|20211010235834_0_...|           1000002|                      |59043

In [25]:
spark.sql("select `_hoodie_commit_time`, trip_id, route_id, destination, tstamp from  hudi_incr_table where trip_id between 999996 and 1000013").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------+--------+-----------+-------------------+
|_hoodie_commit_time|trip_id|route_id|destination|             tstamp|
+-------------------+-------+--------+-----------+-------------------+
|     20211010235834|1000000|       A|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000001|       B|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000002|       C|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000003|       D|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000004|       E|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000005|       F|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000006|       G|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000007|       H|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000008|       I|     Boston|2021-10-10 23:58:01|
|     20211010235834|1000009|       J|     Boston|2021-10-10 23:58:01|
+-------------------+-------+--------+-----------+-------------------+

Now, lets check out point-in-time queries. i.e., query from a specific commit 

In [26]:
commits = spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_trips_table order by commitTime").collect()
print("commits: ")
for elem in commits: print (elem) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

commits: 
Row(commitTime='20211010235153')
Row(commitTime='20211010235834')
Row(commitTime='20211010235951')

In [27]:
#Lets check out the first state. i.e., before we updated the trip to Boston

beginTime = "000" #Represents all commits > this time.
endTime = commits[-3][0] # first commit
print("end time: "+endTime)

# incrementally query data
incViewDF = spark.read.format("org.apache.hudi") \
                   .option(VIEW_TYPE_OPT_KEY, VIEW_TYPE_INCREMENTAL_OPT_VAL) \
                   .option(BEGIN_INSTANTTIME_OPT_KEY, beginTime) \
                   .option(END_INSTANTTIME_OPT_KEY, endTime) \
                   .load(config["target"]) 

incViewDF.show(5)
incViewDF.registerTempTable("hudi_incr_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

end time: 20211010235153
+-------------------+--------------------+------------------+----------------------+--------------------+-------------+--------+-------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|  destination|route_id|trip_id|             tstamp|
+-------------------+--------------------+------------------+----------------------+--------------------+-------------+--------+-------+-------------------+
|     20211010235153|  20211010235153_1_1|           1572203|                      |128bbd02-25e8-4aa...|  Los Angeles|       D|1572203|2021-10-10 23:50:43|
|     20211010235153|  20211010235153_1_2|           1572204|                      |128bbd02-25e8-4aa...|    Las Vegas|       E|1572204|2021-10-10 23:50:43|
|     20211010235153|  20211010235153_1_3|           1572205|                      |128bbd02-25e8-4aa...|       Tucson|       F|1572205|2021-10-10 23:50:43|
|     20211010235153|  2021101023

In [28]:
spark.sql("select `_hoodie_commit_time`, trip_id, route_id, destination, tstamp from  hudi_incr_table where trip_id between 999996 and 1000013").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------+--------+-------------+-------------------+
|_hoodie_commit_time|trip_id|route_id|  destination|             tstamp|
+-------------------+-------+--------+-------------+-------------------+
|     20211010235153|1000000|       A|      Seattle|2021-10-10 23:50:43|
|     20211010235153|1000001|       B|     New York|2021-10-10 23:50:43|
|     20211010235153|1000002|       C|   New Jersey|2021-10-10 23:50:43|
|     20211010235153|1000003|       D|  Los Angeles|2021-10-10 23:50:43|
|     20211010235153|1000004|       E|    Las Vegas|2021-10-10 23:50:43|
|     20211010235153|1000005|       F|       Tucson|2021-10-10 23:50:43|
|     20211010235153|1000006|       G|Washington DC|2021-10-10 23:50:43|
|     20211010235153|1000007|       H| Philadelphia|2021-10-10 23:50:43|
|     20211010235153|1000008|       I|        Miami|2021-10-10 23:50:43|
|     20211010235153|1000009|       J|San Francisco|2021-10-10 23:50:43|
|     20211010235153|1000010|       A|      Seattle

Now we can see that the records from 1000000 to 1000009 display first state of our table before we upserted the trip destination to Boston

### Merge On Read 

The default table type is Copy-On-Write which is best suited for read-heavy workloads with modest writes. Copy-On-Write creates commit files with original data + the new changes during writing itself. While this increases latency on writes, this set up makes it more manageable for faster read.

For near real-time applications that mandate quick upserts, MERGE_ON_READ table type would be better suited. MOR table stores incoming upserts for each file group, onto a row based delta log (In Avro file format). This log is then merged with the existing Parquet file using a a compactor during reads.

In [27]:
## CHANGE ME ##
config = {
    "table_name": "hudi_mor_trips_table",
    "table_name_rt": "hudi_mor_trips_table_rt",
    "target": "s3://vasveena-test-demo/demos/hudi/hudi_mor_trips_table",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "2",
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
STORAGE_TYPE_OPT_KEY="hoodie.datasource.write.storage.type"
COMPACTION_INLINE_OPT_KEY="hoodie.compact.inline"
COMPACTION_MAX_DELTA_COMMITS_OPT_KEY="hoodie.compact.inline.max.delta.commits"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
mor_dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df2 = create_json_df(spark, get_json_data(0, 2000000, mor_dest))
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+-------+-------------------+
|  destination|route_id|trip_id|             tstamp|
+-------------+--------+-------+-------------------+
|      Seattle|       A|      0|2021-10-11 02:09:29|
|     New York|       B|      1|2021-10-11 02:09:29|
|   New Jersey|       C|      2|2021-10-11 02:09:29|
|  Los Angeles|       D|      3|2021-10-11 02:09:29|
|    Las Vegas|       E|      4|2021-10-11 02:09:29|
|       Tucson|       F|      5|2021-10-11 02:09:29|
|Washington DC|       G|      6|2021-10-11 02:09:29|
| Philadelphia|       H|      7|2021-10-11 02:09:29|
|        Miami|       I|      8|2021-10-11 02:09:29|
|San Francisco|       J|      9|2021-10-11 02:09:29|
|      Seattle|       A|     10|2021-10-11 02:09:29|
|     New York|       B|     11|2021-10-11 02:09:29|
|   New Jersey|       C|     12|2021-10-11 02:09:29|
|  Los Angeles|       D|     13|2021-10-11 02:09:29|
|    Las Vegas|       E|     14|2021-10-11 02:09:29|
|       Tucson|       F|     15|2021-10-11 02:

Bulk insert will take the same time as COW as this is the first write 

In [30]:
(df2.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .option(STORAGE_TYPE_OPT_KEY, "MERGE_ON_READ")
      .option(COMPACTION_INLINE_OPT_KEY, "false")
      .option(COMPACTION_MAX_DELTA_COMMITS_OPT_KEY, "1")
      .mode("Overwrite")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Verify the number of files 

Let us check the contents of S3 path. Bulk insert operation on Copy-On-Write and Merge-On-Read tables is identical in terms of performance. 

```
$ aws s3 ls s3://<your-bucket>/demos/hudi/hudi_mor_trips_table/
                           PRE .hoodie/
2021-10-11 00:11:27          0 .hoodie_$folder$
2021-10-11 00:13:05         93 .hoodie_partition_metadata
2021-10-11 00:13:13    4887456 341cc9bf-8cf7-40b6-be2d-8ca55d6c13b6-0_1-186-1671_20211011001125.parquet
2021-10-11 00:13:14    4715858 790fd827-fa27-4748-8e05-c82f3ceeb399-0_0-186-1670_20211011001125.parquet
2021-10-11 00:13:16    4897929 fbe029af-6254-4317-9718-01d37cf3ecdd-0_2-186-1672_20211011001125.parquet
```

Notice the delta commits 

```
$ aws s3 ls s3://<your-bucket>/demos/hudi/hudi_mor_trips_table/.hoodie/
                           PRE .aux/
2021-10-11 00:11:27          0 .aux_$folder$
2021-10-11 00:11:27          0 .temp_$folder$
2021-10-11 00:13:17       3193 20211011001125.deltacommit
2021-10-11 00:12:16          0 20211011001125.deltacommit.inflight
2021-10-11 00:11:29          0 20211011001125.deltacommit.requested
2021-10-11 00:11:27          0 archived_$folder$
2021-10-11 00:11:28        333 hoodie.properties
```

This is the first commit 

Now let us try to upsert some records into MOR table

In [31]:
upsert_dest = ["San Diego", "San Diego", "San Diego", "San Diego", "San Diego","San Diego","San Diego","San Diego","San Diego","San Diego"]
df3 = create_json_df(spark, get_json_data(1000000, 10, upsert_dest))
df3.count()
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+-------+-------------------+
|destination|route_id|trip_id|             tstamp|
+-----------+--------+-------+-------------------+
|  San Diego|       A|1000000|2021-10-11 02:12:58|
|  San Diego|       B|1000001|2021-10-11 02:12:58|
|  San Diego|       C|1000002|2021-10-11 02:12:58|
|  San Diego|       D|1000003|2021-10-11 02:12:58|
|  San Diego|       E|1000004|2021-10-11 02:12:58|
|  San Diego|       F|1000005|2021-10-11 02:12:58|
|  San Diego|       G|1000006|2021-10-11 02:12:58|
|  San Diego|       H|1000007|2021-10-11 02:12:58|
|  San Diego|       I|1000008|2021-10-11 02:12:58|
|  San Diego|       J|1000009|2021-10-11 02:12:58|
+-----------+--------+-------+-------------------+

In [32]:
(df3.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .option(STORAGE_TYPE_OPT_KEY, "MERGE_ON_READ")
      .option(COMPACTION_INLINE_OPT_KEY, "false")
      .option(COMPACTION_MAX_DELTA_COMMITS_OPT_KEY, "1")
      .mode("Append")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name']+ "_ro where trip_id between 999996 and 1000010").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|999996 |G       |Washington DC|2021-10-11 02:09:29|
|999997 |H       |Philadelphia |2021-10-11 02:09:29|
|999998 |I       |Miami        |2021-10-11 02:09:29|
|999999 |J       |San Francisco|2021-10-11 02:09:29|
|1000000|A       |Seattle      |2021-10-11 02:09:29|
|1000001|B       |New York     |2021-10-11 02:09:29|
|1000002|C       |New Jersey   |2021-10-11 02:09:29|
|1000003|D       |Los Angeles  |2021-10-11 02:09:29|
|1000004|E       |Las Vegas    |2021-10-11 02:09:29|
|1000005|F       |Tucson       |2021-10-11 02:09:29|
|1000006|G       |Washington DC|2021-10-11 02:09:29|
|1000007|H       |Philadelphia |2021-10-11 02:09:29|
|1000008|I       |Miami        |2021-10-11 02:09:29|
|1000009|J       |San Francisco|2021-10-11 02:09:29|
|1000010|A       |Seattle      |2021-10-11 02:09:29|
+-------+--------+-------------+--------------

Lets query the real-time table

In [34]:
spark.sql("select trip_id, route_id, destination, tstamp from "+ config['table_name_rt'] + " where trip_id between 999996 and 1000010").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|999996 |G       |Washington DC|2021-10-11 02:09:29|
|999997 |H       |Philadelphia |2021-10-11 02:09:29|
|999998 |I       |Miami        |2021-10-11 02:09:29|
|999999 |J       |San Francisco|2021-10-11 02:09:29|
|1000000|A       |San Diego    |2021-10-11 02:12:58|
|1000001|B       |San Diego    |2021-10-11 02:12:58|
|1000002|C       |San Diego    |2021-10-11 02:12:58|
|1000003|D       |San Diego    |2021-10-11 02:12:58|
|1000004|E       |San Diego    |2021-10-11 02:12:58|
|1000005|F       |San Diego    |2021-10-11 02:12:58|
|1000006|G       |San Diego    |2021-10-11 02:12:58|
|1000007|H       |San Diego    |2021-10-11 02:12:58|
|1000008|I       |San Diego    |2021-10-11 02:12:58|
|1000009|J       |San Diego    |2021-10-11 02:12:58|
|1000010|A       |Seattle      |2021-10-11 02:09:29|
+-------+--------+-------------+--------------

Check the S3 path again. There is no change in number of Parquet files after upsert operation unlike Copy-On-Write tables. 

```
$  aws s3 ls s3://vasveena-test-demo/demos/hudi/hudi_mor_trips_table/
                           PRE .hoodie/
2021-10-11 02:13:07       2071 .1dc7e87f-2160-4a6b-b3b4-3c89f6170261-0_20211011021111.log.1_0-197-1880
2021-10-11 02:11:12          0 .hoodie_$folder$
2021-10-11 02:12:41         93 .hoodie_partition_metadata
2021-10-11 02:12:49    4761546 1dc7e87f-2160-4a6b-b3b4-3c89f6170261-0_0-163-1551_20211011021111.parquet
2021-10-11 02:12:48    4648132 49e19858-1ae8-493b-9bbe-06d90ec74983-0_2-163-1553_20211011021111.parquet
2021-10-11 02:12:50    5198196 578042a0-fe9d-4c2d-adfc-310a06b78494-0_1-163-1552_20211011021111.parquet

```

Now, let us compact using Hudi CLI 

On EMR:

```
/usr/lib/hudi/cli/bin/hudi-cli.sh

hudi->connect --path s3://<your-bucket>/demos/hudi/hudi_mor_trips_table/

hudi:hudi_mor_trips_table->compactions show all
╔═════════════════════════╤═══════╤═══════════════════════════════╗
║ Compaction Instant Time │ State │ Total FileIds to be Compacted ║
╠═════════════════════════╧═══════╧═══════════════════════════════╣
║ (empty)                                                         ║
╚═════════════════════════════════════════════════════════════════╝

```

Notice there are no compactions for our mor table. Let us manually schedule and run a compaction. 

```

hudi:hudi_mor_trips_table->compaction schedule --hoodieConfigs hoodie.datasource.hive_sync.partition_extractor_class=com.uber.hoodie.NonpartitionedKeyGenerator,hoodie.compact.inline.max.delta.commits=1

Attempted to schedule compaction for 20211011021355


hudi:hudi_mor_trips_table->connect --path s3://<your-bucket>/demos/hudi/hudi_mor_trips_table/

hudi:hudi_mor_trips_table->compactions show all
20/04/28 23:39:04 INFO timeline.HoodieActiveTimeline: Loaded instants java.util.stream.ReferencePipeline$Head@240d2f9d
20/04/28 23:39:04 INFO s3n.S3NativeFileSystem: Opening 's3://<Your S3 Bucket Here>/demos/hudi/hudi_mor_trips_table/.hoodie/.aux/20211011021355.compaction.requested' for reading
╔═════════════════════════╤═══════════╤═══════════════════════════════╗
║ Compaction Instant Time │ State     │ Total FileIds to be Compacted ║
╠═════════════════════════╪═══════════╪═══════════════════════════════╣
║ 20211011021355          │ REQUESTED │ 1                             ║
╚═════════════════════════╧═══════════╧═══════════════════════════════╝

hudi:hudi_mor_trips_table->compaction run --parallelism 10 --sparkMemory 4G --retry 2 --schemaFilePath s3://<your-bucket>/demos/schema.avsc --compactionInstant 20211011021355

Compaction successfully completed for 20211011021355

```

You can also schedule a compaction using "compaction schedule" option. 

Now, if we check the S3 path, we will see the delta commit has been merged into a new file 

```
$  aws s3 ls s3://vasveena-test-demo/demos/hudi/hudi_mor_trips_table/
                           PRE .hoodie/
2021-10-11 02:13:07       2071 .1dc7e87f-2160-4a6b-b3b4-3c89f6170261-0_20211011021111.log.1_0-197-1880
2021-10-11 02:11:12          0 .hoodie_$folder$
2021-10-11 02:12:41         93 .hoodie_partition_metadata
2021-10-11 02:17:42    4760914 1dc7e87f-2160-4a6b-b3b4-3c89f6170261-0_0-0-0_20211011021355.parquet
2021-10-11 02:12:49    4761546 1dc7e87f-2160-4a6b-b3b4-3c89f6170261-0_0-163-1551_20211011021111.parquet
2021-10-11 02:12:48    4648132 49e19858-1ae8-493b-9bbe-06d90ec74983-0_2-163-1553_20211011021111.parquet
2021-10-11 02:12:50    5198196 578042a0-fe9d-4c2d-adfc-310a06b78494-0_1-163-1552_20211011021111.parquet

```


## Working with Partitioned Tables

Let's do the same thing with Partitioned Tables. For the sake of this demo, we will be making route_id as partition field. You can also have a nested partition structure like yyyy/mm/dd which is more common

In [35]:
## CHANGE ME ##
config = {
    "table_name": "hudi_partitioned_trips_table",
    "target": "s3://vasveena-test-demo/demos/hudi/hudi_partitioned_trips_table",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "2",
    "partition_keys" : "route_id"
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Let's generate the data:

In [36]:
part_dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df1 = create_json_df(spark, get_json_data(0, 2000000, part_dest))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We add the partitionKey column to the dataframe.

In [37]:
from pyspark.sql.functions import concat, col, lit

hudiTablePartitionKey="route_id"
df1 = df1.withColumn(hudiTablePartitionKey,concat(lit("route_id="),col("route_id")))
df1.select(hudiTablePartitionKey).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|  route_id|
+----------+
|route_id=A|
|route_id=B|
|route_id=C|
|route_id=D|
|route_id=E|
+----------+
only showing top 5 rows

And we can now write out the data to S3. Notice that the Hive Partition Extractor class has changed in the statement below:

```
      .option(HIVE_PARTITION_FIELDS_OPT_KEY, config["partition_keys"])
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL)
      .option(PARTITIONPATH_FIELD_OPT_KEY,"route_id")
```


In [38]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 6)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_PARTITION_FIELDS_OPT_KEY, config["partition_keys"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL)
      .option(PARTITIONPATH_FIELD_OPT_KEY,"route_id")
      .mode("Overwrite")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We can see the partitions fields are present in our Hive table from Glue catalog

```
PARTITIONED BY (`route_id` STRING)
```

Let's now query the data and group by the the partition columns:

In [39]:
spark.sql("select route_id, count(*) as num_trips from "+config['table_name']+" group by route_id order by route_id").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+
|route_id|num_trips|
+--------+---------+
|A       |200000   |
|B       |200000   |
|C       |200000   |
|D       |200000   |
|E       |200000   |
|F       |200000   |
|G       |200000   |
|H       |200000   |
|I       |200000   |
|J       |200000   |
+--------+---------+

Let us check the S3 path

```
$ $ aws s3 ls s3://vasveena-test-demo/demos/hudi/hudi_partitioned_trips_table/
                           PRE .hoodie/
                           PRE route_id=A/
                           PRE route_id=B/
                           PRE route_id=C/
                           PRE route_id=D/
                           PRE route_id=E/
                           PRE route_id=F/
                           PRE route_id=G/
                           PRE route_id=H/
                           PRE route_id=I/
                           PRE route_id=J/
2021-10-11 02:19:21          0 .hoodie_$folder$
2021-10-11 02:21:07          0 route_id=A_$folder$
2021-10-11 02:21:06          0 route_id=B_$folder$
2021-10-11 02:21:08          0 route_id=C_$folder$
2021-10-11 02:21:07          0 route_id=D_$folder$
2021-10-11 02:21:10          0 route_id=E_$folder$
2021-10-11 02:21:06          0 route_id=F_$folder$
2021-10-11 02:21:06          0 route_id=G_$folder$
2021-10-11 02:21:09          0 route_id=H_$folder$
2021-10-11 02:21:07          0 route_id=I_$folder$
2021-10-11 02:21:10          0 route_id=J_$folder$

```

Under each partition, there will be partition metadata

```

$ aws s3 ls s3://vasveena-test-demo/demos/hudi/hudi_partitioned_trips_table/route_id=A/
2021-10-11 02:21:07         93 .hoodie_partition_metadata
2021-10-11 02:21:10    1723511 0e3f96bc-f05e-46a9-912b-156dccad1559-0_0-222-2111_20211011021919.parquet

```

The other operations Upsert etc. behave the same way on Partitioned tables.